In [1]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [30]:
counters = sqlContext.read.format("org.apache.spark.sql.cassandra").load(table="counters", keyspace="dice")
il = sqlContext.read.format("org.apache.spark.sql.cassandra").load(table="inductive_loops", keyspace="dice")

In [31]:
counters.printSchema()
il.printSchema()

root
 |-- id: string (nullable = true)
 |-- modified: timestamp (nullable = true)
 |-- contentname: string (nullable = true)
 |-- crsid: string (nullable = true)
 |-- description: string (nullable = true)
 |-- icon: string (nullable = true)
 |-- stevci_cestaopis: string (nullable = true)
 |-- stevci_gap: float (nullable = true)
 |-- stevci_hit: integer (nullable = true)
 |-- stevci_lokacijaopis: string (nullable = true)
 |-- stevci_pasopis: string (nullable = true)
 |-- stevci_smeropis: string (nullable = true)
 |-- stevci_stat: integer (nullable = true)
 |-- stevci_statopis: string (nullable = true)
 |-- stevci_stev: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- x: float (nullable = true)
 |-- x_wgs: float (nullable = true)
 |-- y: float (nullable = true)
 |-- y_wgs: float (nullable = true)

root
 |-- id: string (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- avgspeed: integer (nullable = true)
 |-- chainage: integer (nullable = true)
 |-- date: 

In [32]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

statTokens = udf(lambda stat: 'Heavy traffic' if stat=='Gost promet' else 'Traffic congestion', StringType())
counters = counters.withColumn('stat', statTokens(counters.stevci_statopis))
laneTokens = udf(lambda lane: '(d)' if lane=='(v)' else '(o)', StringType())
counters = counters.withColumn('lane', laneTokens(counters.stevci_pasopis))

il = il.fillna({'lanedescription': ' '})

In [33]:
from pyspark.sql.functions import col

counters = counters.where('stevci_stat==4 or stevci_stat=5').select('id', col('stevci_lokacijaopis').alias('location'), col('stevci_cestaopis').alias('road'), col('stevci_smeropis').alias('direction'), 'lane', 'stat', col('x_wgs').alias('longitude'), col('y_wgs').alias('latitude'), col('modified').alias('datetime'))
il = il.where('stat==4 or stat=5').select('id', col('locationDescription').alias('location'), col('roadDescription').alias('road'), col('directionDescription').alias('direction'), col('laneDescription').alias('lane'), col('StatusDescription').alias('stat'), col('deviceY').alias('longitude'), col('deviceX').alias('latitude'), col('updated').alias('datetime'))

In [45]:
all_traffic = counters.unionAll(il)
all_traffic.printSchema()

root
 |-- id: string (nullable = true)
 |-- location: string (nullable = true)
 |-- road: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- lane: string (nullable = true)
 |-- stat: string (nullable = true)
 |-- longitude: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- datetime: timestamp (nullable = true)



In [44]:
counters.write.format("org.apache.spark.sql.cassandra").mode('append').options(table="heavy_traffic", keyspace="dice_results").save()